<a href="https://colab.research.google.com/github/profteachkids/StemUnleashed/blob/main/LagrangeCollocationODE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
import jax.numpy as jnp
import jax
import numpy as np
jax.config.update('jax_enable_x64',True)
from plotly.subplots import make_subplots
from scipy.special import roots_jacobi

In [81]:
N=7
xs=np.r_[-1.,roots_jacobi(N-2,0,1)[0],1.]

non_diag=np.logical_not(np.eye(N).astype(bool))
xm = np.tile(xs, (N,1))[non_diag].reshape(N,-1)
xj = np.tile(xs.reshape(N,1), N-1)
xj_xm = 1/(xj-xm)


def create_lagrange(h=2):

    def lj(x):
        m=(x-xm) * xj_xm
        return jnp.prod(m,axis=1)

    lj.grad=jax.jacobian(lj)
    lj.gradvec = jax.vmap(lj.grad)

    lj.grad2=jax.jacobian(lj.grad)
    lj.grad2vec = jax.vmap(lj.grad2)

    lj.yps = lj.gradvec(xs)
    lj.yp2s = lj.grad2vec(xs)

    def yp(ys):
        return (lj.yps @ ys)*(2/h)
    lj.yp=yp

    def yp2(ys):
        return (lj.yp2s @ ys)*(2/h)**2
    lj.yp=yp
    lj.yp2=yp2

    return lj

In [82]:
def x(a,b):
    return (b-a)/2*xs + (a+b)/2

ys = jnp.sin(x(0,0.2))

In [83]:
jnp.cos(x(0,0.2))

Array([1.        , 0.99980582, 0.99814573, 0.9936892 , 0.98716391,
       0.9816173 , 0.98006658], dtype=float64)

In [84]:
lj=create_lagrange(h=0.2)
lj.yp(ys)

Array([1.        , 0.99980582, 0.99814573, 0.9936892 , 0.98716391,
       0.9816173 , 0.98006658], dtype=float64)

In [85]:
lj.yp2(ys)

Array([-3.03699288e-08, -1.97057412e-02, -6.08694950e-02, -1.12168483e-01,
       -1.59710436e-01, -1.90859852e-01, -1.98669321e-01], dtype=float64)

In [86]:
-jnp.sin(x(0,0.2))

Array([-0.        , -0.01970574, -0.06086949, -0.11216848, -0.15971044,
       -0.19085985, -0.19866933], dtype=float64)